## Segmenting and Clustering Neighborhoods in Toronto

In [ ]:
# Installing necessary packages
!pip install BeautifulSoup4
!pip install requests

- Getting Data from wikipedia
- parsing html data using BeautifulSoup
- storing the data into Pandas DataFrame

In [2]:
#imports
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [13]:
#pulling data from wiki page and creating soup object
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(source.text, 'html.parser')

#iterating to get the data from htm page and store it into a list
data = []
columns = []
table = soup.find(class_='wikitable')
for index, tr in enumerate(table.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
        section.append(td.text.rstrip())
        
    if(index == 0):
        columns = section
    else:
        data.append(section)

#converting the available data into Pandas DataFrame
toronto_df = pd.DataFrame(data = data, columns = columns)
toronto_df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


- Removing the 'Not assigned' in Boroughts

In [14]:
#Remove Boroughts that are 'Not assigned'
toronto_df = toronto_df[toronto_df['Borough'] != 'Not assigned']
toronto_df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


- More than one neighborhood can exist in one postal code area, combined these into one row with the neighborhoods separated with a comma
- If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [15]:
toronto_df["Neighborhood"] = toronto_df.groupby("Postal Code")["Neighborhood"].transform(lambda neigh: ','.join(neigh))

toronto_df = toronto_df.drop_duplicates()

if(toronto_df.index.name != 'Postal Code'):
    toronto_df = toronto_df.set_index('Postal Code')

toronto_df.head()

,Borough,Neighborhood
Postal Code,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Regent Park, Harbourfront"
M6A,North York,"Lawrence Manor, Lawrence Heights"
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [16]:
toronto_df['Neighborhood'].replace("Not assigned",toronto_df["Borough"], inplace=True)
toronto_df.head()

,Borough,Neighborhood
Postal Code,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Regent Park, Harbourfront"
M6A,North York,"Lawrence Manor, Lawrence Heights"
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


- In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe

In [9]:
toronto_df.shape

(103, 2)

### Adding Geospatial data

In [17]:
#loading data lat/lon data from the csv file
lat_lng_coords_df = pd.read_csv("Geospatial_Coordinates.csv")

#converting index to postal code
lat_lng_coords_df.columns = ["Postal Code", "Latitude", "Longitude"]
if(lat_lng_coords_df.index.name != 'Postal Code'):
    lat_lng_coords_df = lat_lng_coords_df.set_index('Postal Code')
    
lat_lng_coords_df.head()

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


In [18]:
toronto_df = toronto_df.join(lat_lng_coords_df)
toronto_df.head(11)

,Borough,Neighborhood,Latitude,Longitude
Postal Code,,,,
M3A,North York,Parkwoods,43.753259,-79.329656
M4A,North York,Victoria Village,43.725882,-79.315572
M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
M3B,North York,Don Mills,43.745906,-79.352188
M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
